# Séance 1 - Objectifs, chargement, traitement

## Étape 1 - Choix de la source de données

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("Seance2")
  .master("local[*]")
  .getOrCreate()

lastException = null
spark = org.apache.spark.sql.SparkSession@58384a02


org.apache.spark.sql.SparkSession@58384a02

In [3]:
val lignes = sc.textFile("data/Trajets_train.csv")
lignes.take(10).foreach(println)

id,ville_depart,ville_arrivee,distance_km,duree_min
1,Nice,Lyon,470,188
2,Dijon,Tours,290,116
3,Marseille,Lyon,315,126
4,Nantes,Bordeaux,335,134
5,Grenoble,Dijon,309,124
6,Nantes,Dijon,323,129
7,Lille,Dijon,468,187
8,Dijon,Strasbourg,310,124
9,Nantes,Lyon,516,206


lignes = data/Trajets_train.csv MapPartitionsRDD[3] at textFile at <console>:25


data/Trajets_train.csv MapPartitionsRDD[3] at textFile at <console>:25

## Étape 2 – Persona

**Type d’utilisateur :**  
Agent de planification des transports (exemple : agent SNCF)

**Persona :**  
- **Nom :** Julien Dubois  
- **Métier :** Agent SNCF  
- **Objectif :** Améliorer les trajets entre les grandes villes

**Indicateurs pertinents :**
- Temps moyen de trajet par liaison (ville_depart → ville_arrivee)
- Distance moyenne par liaison
- Vitesse moyenne (km/h) par liaison
- Les trajets les plus lents (vitesse basse)

## Étape 3 - Chargement, pré-traitement

In [4]:

// Définition de la structure d’un trajet avec une case class
case class Trajet(
  id: Int,                 // Identifiant unique du trajet
  ville_depart: String,    // Ville de départ
  ville_arrivee: String,   // Ville d’arrivée
  distance_km: Int,        // Distance en kilomètres
  duree_min: Int           // Durée du trajet en minutes
)

val lignesSansEntete = lignes.filter(!_.startsWith("id"))

// Transformation de chaque ligne du CSV en un objet Trajet
val trajets = lignesSansEntete.map { ligne =>
  val champs = ligne.split(",")  // Découper la ligne en colonnes (tableau de chaînes)
  Trajet(
    champs(0).toInt,   // id
    champs(1),         // ville_depart
    champs(2),         // ville_arrivee
    champs(3).toInt,   // distance_km
    champs(4).toInt    // duree_min
  )
}

trajets.take(5).foreach(println)


Trajet(1,Nice,Lyon,470,188)
Trajet(2,Dijon,Tours,290,116)
Trajet(3,Marseille,Lyon,315,126)
Trajet(4,Nantes,Bordeaux,335,134)
Trajet(5,Grenoble,Dijon,309,124)


defined class Trajet
lignesSansEntete = MapPartitionsRDD[4] at filter at <console>:34
trajets = MapPartitionsRDD[5] at map at <console>:37


MapPartitionsRDD[5] at map at <console>:37

# Séance 2 - Ingestion, SparkSQL

## Étape 1 : Transformer le RDD en DataFrame SparkSQL

In [5]:
import spark.implicits._

// Transformer le RDD en DataFrame
val dfTrajets = trajets.toDF()

println("Nombre de trajets : " + dfTrajets.count())

dfTrajets.show()

dfTrajets.printSchema()

// Statistiques résumées sur toutes les colonnes numériques
dfTrajets.summary().show()

Nombre de trajets : 100
+---+------------+-------------+-----------+---------+
| id|ville_depart|ville_arrivee|distance_km|duree_min|
+---+------------+-------------+-----------+---------+
|  1|        Nice|         Lyon|        470|      188|
|  2|       Dijon|        Tours|        290|      116|
|  3|   Marseille|         Lyon|        315|      126|
|  4|      Nantes|     Bordeaux|        335|      134|
|  5|    Grenoble|        Dijon|        309|      124|
|  6|      Nantes|        Dijon|        323|      129|
|  7|       Lille|        Dijon|        468|      187|
|  8|       Dijon|   Strasbourg|        310|      124|
|  9|      Nantes|         Lyon|        516|      206|
| 10|       Rouen|        Dijon|        157|       63|
| 11|       Dijon|        Tours|        515|      206|
| 12|  Strasbourg|        Tours|        562|      225|
| 13|        Lyon|         Nice|        290|      116|
| 14|       Lille|        Tours|        263|      105|
| 15|      Angers|       Rennes|        2

dfTrajets = [id: int, ville_depart: string ... 3 more fields]


[id: int, ville_depart: string ... 3 more fields]

## Étape 2 - Extraction de dimensions

In [5]:
import org.apache.spark.sql.functions.monotonically_increasing_id

// Dimension : villes de départ
val dfVillesDepart = dfTrajets
  .select("ville_depart")
  .distinct()
  .withColumn("id_ville_depart", monotonically_increasing_id())

// Dimension : villes d’arrivée
val dfVillesArrivee = dfTrajets
  .select("ville_arrivee")
  .distinct()
  .withColumn("id_ville_arrivee", monotonically_increasing_id())

// Dimension : modes de transport
val dfModesTransport = dfTrajets
  .select("mode_transport")
  .distinct()
  .withColumn("id_mode_transport", monotonically_increasing_id())

// Ajouter l’ID de la ville de départ
val dfTrajetsJoin1 = dfTrajets
  .join(dfVillesDepart, Seq("ville_depart"))

// Ajouter l’ID de la ville d’arrivée
val dfTrajetsJoin2 = dfTrajetsJoin1
  .join(dfVillesArrivee, Seq("ville_arrivee"))

// Ajouter l’ID du mode de transport
val dfTrajetsFinal = dfTrajetsJoin2
  .join(dfModesTransport, Seq("mode_transport"))

val dfTrajetsClean = dfTrajetsFinal
  .drop("ville_depart")
  .drop("ville_arrivee")
  .drop("mode_transport")

dfTrajetsClean.show()



+---+-----------+---------+---------------+----------------+-----------------+
| id|distance_km|duree_min|id_ville_depart|id_ville_arrivee|id_mode_transport|
+---+-----------+---------+---------------+----------------+-----------------+
|  1|        266|       57|              0|               8|                1|
| 23|        516|      191|              0|               9|                1|
| 37|        439|      370|              0|               4|                0|
|  2|        290|      706|              1|              13|                2|
|  8|        592|      351|              1|              10|                1|
| 11|        515|     1989|              1|              13|                2|
| 30|        409|       99|              1|               6|                1|
| 42|         56|       58|              1|               6|                0|
| 47|        145|      463|              1|               2|                2|
|  5|        309|       87|              2|         

dfVillesDepart = [ville_depart: string, id_ville_depart: bigint]
dfVillesArrivee = [ville_arrivee: string, id_ville_arrivee: bigint]
dfModesTransport = [mode_transport: string, id_mode_transport: bigint]
dfTrajetsJoin1 = [ville_depart: string, id: int ... 5 more fields]
dfTrajetsJoin2 = [ville_arrivee: string, ville_depart: string ... 6 more fields]
dfTrajetsFinal = [mode_transport: string, ville_arrivee: string ... 7 more fields]
dfTrajetsClean = [id: int, distance_km: int ... 4 more fields]


[id: int, distance_km: int ... 4 more fields]

## Étape 3 - Tables Hive, SQL

In [10]:
import org.apache.spark.sql.hive.HiveContext

val hc = new HiveContext(sc)

 dfTrajets.write.mode("overwrite").saveAsTable("trajets")
 dfVillesDepart.write.mode("overwrite").saveAsTable("villes_depart")
 dfVillesArrivee.write.mode("overwrite").saveAsTable("villes_arrivee")
 dfModesTransport.write.mode("overwrite").saveAsTable("modes_transport")

// 1. Tous les trajets en train
hc.sql("SELECT * FROM trajets WHERE mode_transport = 'train'").show()

// 2. Nombre de trajets par mode de transport
hc.sql("SELECT mode_transport, COUNT(*) as nb_trajets FROM trajets GROUP BY mode_transport").show()

// 3. Distance moyenne des trajets en voiture
hc.sql("SELECT AVG(distance_km) as distance_moyenne FROM trajets WHERE mode_transport = 'voiture'").show()


+---+------------+-------------+-----------+--------------+---------+
| id|ville_depart|ville_arrivee|distance_km|mode_transport|duree_min|
+---+------------+-------------+-----------+--------------+---------+
|  1|        Nice|         Lyon|        266|         train|       57|
|  5|    Grenoble|        Dijon|        309|         train|       87|
|  8|       Dijon|   Strasbourg|        592|         train|      351|
| 15|      Angers|       Rennes|        298|         train|       78|
| 23|        Nice|     Bordeaux|        516|         train|      191|
| 24|       Rouen|        Lille|         80|         train|       19|
| 30|       Dijon|    Marseille|        409|         train|       99|
| 31|    Bordeaux|        Lille|         42|         train|       13|
| 32|       Paris|   Strasbourg|        228|         train|      128|
| 33|    Grenoble|        Tours|         82|         train|       30|
| 40|    Grenoble|        Lille|        557|         train|      306|
| 51|       Lille|  

lastException = null
hc = org.apache.spark.sql.hive.HiveContext@3e4baca9


org.apache.spark.sql.hive.HiveContext@3e4baca9

# Séance 3 - Spark SQL (suite), Stats

## Étape 1 - Agrégations

### 1. Moyenne des vitesses par mode de transport

In [12]:
hc.sql("""
  SELECT 
    mode_transport,
    ROUND(AVG(distance_km * 60.0 / duree_min), 2) AS vitesse_moyenne_kmh
  FROM trajets
  GROUP BY mode_transport
""").show()


+--------------+-------------------+
|mode_transport|vitesse_moyenne_kmh|
+--------------+-------------------+
|           bus|              70.09|
|         train|             197.86|
|          velo|              18.50|
|       voiture|              92.25|
+--------------+-------------------+



### 2. Nombre de trajets par mode de transport

In [13]:
hc.sql("""
  SELECT 
    mode_transport,
    COUNT(*) AS nb_trajets
  FROM trajets
  GROUP BY mode_transport
""").show()

+--------------+----------+
|mode_transport|nb_trajets|
+--------------+----------+
|           bus|        22|
|         train|        24|
|          velo|        24|
|       voiture|        30|
+--------------+----------+



### 3. Moyenne de la distance par mode de transport

In [14]:
hc.sql("""
  SELECT 
    mode_transport,
    ROUND(AVG(distance_km), 1) AS distance_moyenne_km
  FROM trajets
  GROUP BY mode_transport
""").show()


+--------------+-------------------+
|mode_transport|distance_moyenne_km|
+--------------+-------------------+
|           bus|              371.7|
|         train|              392.6|
|          velo|              304.9|
|       voiture|              405.9|
+--------------+-------------------+



### 4. Top 5 des trajets en train les plus lents (en vitesse km/h)

In [18]:
hc.sql("""
  SELECT 
    ville_depart,
    ville_arrivee,
    distance_km,
    duree_min,
    ROUND(distance_km * 60.0 / duree_min, 2) AS vitesse_kmh
  FROM trajets
  WHERE mode_transport = 'train'
  ORDER BY vitesse_kmh ASC
  LIMIT 5
""").show()

+------------+-------------+-----------+---------+-----------+
|ville_depart|ville_arrivee|distance_km|duree_min|vitesse_kmh|
+------------+-------------+-----------+---------+-----------+
|       Dijon|   Strasbourg|        592|      351|     101.20|
|       Paris|   Strasbourg|        228|      128|     106.88|
|    Grenoble|        Lille|        557|      306|     109.22|
|       Paris|    Marseille|        150|       82|     109.76|
|      Rennes|       Angers|        689|      315|     131.24|
+------------+-------------+-----------+---------+-----------+



## 5. Trajets en train avec une durée anormalement élevée (ratio minutes/km élevé)

In [19]:
hc.sql("""
  SELECT 
    ville_depart,
    ville_arrivee,
    distance_km,
    duree_min,
    ROUND(duree_min * 1.0 / distance_km, 2) AS min_par_km
  FROM trajets
  WHERE mode_transport = 'train' AND distance_km > 50
  ORDER BY min_par_km DESC
  LIMIT 5
""").show()

+------------+-------------+-----------+---------+----------+
|ville_depart|ville_arrivee|distance_km|duree_min|min_par_km|
+------------+-------------+-----------+---------+----------+
|       Dijon|   Strasbourg|        592|      351|      0.59|
|       Paris|   Strasbourg|        228|      128|      0.56|
|       Paris|    Marseille|        150|       82|      0.55|
|    Grenoble|        Lille|        557|      306|      0.55|
|      Rennes|       Angers|        689|      315|      0.46|
+------------+-------------+-----------+---------+----------+



## 6. Nombre de trajets par ville de départ

In [17]:
hc.sql("""
  SELECT 
    ville_depart,
    COUNT(*) AS nb_trajets_depart
  FROM trajets
  GROUP BY ville_depart
  ORDER BY nb_trajets_depart DESC
""").show()

+------------+-----------------+
|ville_depart|nb_trajets_depart|
+------------+-----------------+
|  Strasbourg|               11|
|       Dijon|               10|
|        Lyon|               10|
|       Lille|                9|
|    Grenoble|                8|
|       Paris|                7|
|    Bordeaux|                7|
|       Rouen|                7|
|      Nantes|                6|
|        Nice|                5|
|      Angers|                5|
|      Rennes|                5|
|    Toulouse|                4|
|   Marseille|                3|
|       Tours|                3|
+------------+-----------------+

